In [8]:
import pandas as pd
import os
import re

In [9]:
# 1. Process degree_streams.csv to create a degree-to-stream map
degree_map_df = pd.read_csv('degree_streams.csv')
degree_map_df.rename(columns={'Stream': 'stream', 'Degree': 'degree_name', 'Universities': 'university'}, inplace=True)

# Clean the stream names and convert degree names to uppercase
degree_map_df['stream'] = degree_map_df['stream'].str.replace(' Stream', '').str.lower()
degree_map_df['degree_name'] = degree_map_df['degree_name'].str.upper().str.strip()

print("Processed Degree-to-Stream Map Head:")
degree_map_df.head()

Processed Degree-to-Stream Map Head:


,stream,degree_name
0,arts,ARTS
1,arts,ARTS (SP)
2,arts,ARTS (SAB)
3,arts,COMMUNICATION STUDIES
4,arts,PEACE AND CONFLICT RESOLUTION


In [10]:
# 2. Process the performance data from the 'perfs' directory
def process_perfs_data(file_path):
    df = pd.read_csv(file_path)
    file_name = os.path.basename(file_path)
    parts = file_name.replace('.csv', '').split('_')
    df['stream'] = ' '.join(parts[1:-1])
    df['year'] = int(parts[-1])
    return df

perfs_files = [os.path.join('perfs', f) for f in os.listdir('perfs') if f.endswith('.csv')]
all_perfs_data = pd.concat([process_perfs_data(f) for f in perfs_files], ignore_index=True)
all_perfs_data.rename(columns={'No. Sat': 'students_sat', 'Eligible for University Entrance': 'students_passed'}, inplace=True)
all_perfs_data = all_perfs_data[['District', 'students_sat', 'students_passed', 'stream', 'year']]
all_perfs_data = all_perfs_data[all_perfs_data['District'].str.lower() != 'all island'].copy()

print("Processed Performance Data Head:")
all_perfs_data.head()

Processed Performance Data Head:


,District,students_sat,students_passed,stream,year
0,Batticaloa,898,654,commerce,2021
1,Jaffna,949,685,commerce,2021
2,Puttalam,1654,1178,commerce,2021
3,Monaragala,1210,852,commerce,2021
4,Kurunegala,3688,2583,commerce,2021


In [11]:
# 3. Process the z-score cutoff data from the 'cops' directory
def process_cops_data(file_path):
    df = pd.read_csv(file_path)
    df.rename(columns={df.columns[0]: 'District'}, inplace=True)
    year_match = re.search(r'(\d{4})', file_path)
    if year_match:
        df['year'] = int(year_match.group(1))
    else:
        raise ValueError(f"Could not extract year from {file_path}")
    return df.melt(id_vars=['District', 'year'], var_name='degree', value_name='z-score_cutoff')

cops_files = [os.path.join('cops', f) for f in os.listdir('cops') if f.startswith('cop_') and f.endswith('.csv')]
all_cops_data = pd.concat([process_cops_data(f) for f in cops_files], ignore_index=True)

print("Processed Z-Score Data Head:")
all_cops_data.head()

Processed Z-Score Data Head:


,District,year,degree,z-score_cutoff
0,COLOMBO,2019,MEDICINE University of Colombo,2.4546
1,GAMPAHA,2019,MEDICINE University of Colombo,2.501
2,KALUTARA,2019,MEDICINE University of Colombo,2.5199
3,MATALE,2019,MEDICINE University of Colombo,2.5115
4,KANDY,2019,MEDICINE University of Colombo,2.5805


In [12]:
# 4. Add the 'stream' to the z-score data with robust name extraction

def extract_degree_name_from_string(degree_string):
    words = str(degree_string).split()
    degree_name_parts = []
    for word in words:
        # A word is part of the degree name if it is all uppercase.
        # We also allow ampersands and hyphens.
        if word.isupper() or word in ['&', '-']:
            degree_name_parts.append(word)
        else:
            # The first word not in all caps marks the start of the university name.
            break
    return ' '.join(degree_name_parts).strip()

all_cops_data['degree_name'] = all_cops_data['degree'].apply(extract_degree_name_from_string)

# The degree_map_df already has the 'degree_name' in uppercase
stream_map = degree_map_df[['degree_name', 'stream']].drop_duplicates()

all_cops_data_with_stream = pd.merge(all_cops_data, stream_map, on='degree_name', how='left')

print(f"Rows in cops data: {len(all_cops_data)}")
print(f"Rows with a matched stream: {all_cops_data_with_stream['stream'].notna().sum()}")
print(f"Rows MISSING a stream: {all_cops_data_with_stream['stream'].isna().sum()}")

if all_cops_data_with_stream['stream'].isna().any():
    unmatched = all_cops_data_with_stream[all_cops_data_with_stream['stream'].isna()]['degree'].value_counts()
    print("Top 5 Unmatched Degrees:")
    print(unmatched.head())

Rows in cops data: 25450
Rows with a matched stream: 22225
Rows MISSING a stream: 3225
Top 5 Unmatched Degrees:
degree
PEACE & CONFLICT RESOLUTION  University of Kelaniya                     125
TOWN & COUNTRY PLANNING University of Moratuwa                          125
AGRI BUSINESS MANAGEMENT University of Ruhuna                           125
MARINE AND FRESHWATER SCIENCES University of Ruhuna                     125
COMPUTING & INFORMATION SYSTEMS Sabaragamuwa University of Sri Lanka    125
Name: count, dtype: int64


In [13]:
# 5. Perform the final merge and save the dataset

final_cops_data = all_cops_data_with_stream.dropna(subset=['stream']).copy()

# Standardize the 'District' column in both dataframes
final_cops_data['District'] = final_cops_data['District'].str.strip().str.lower()
all_perfs_data['District'] = all_perfs_data['District'].str.strip().str.lower()

# Merge on the correct composite key
merged_df = pd.merge(final_cops_data, all_perfs_data, on=['District', 'year', 'stream'])

# Clean the z-score data
final_df = merged_df[merged_df['z-score_cutoff'] != 'NQC'].copy()
final_df['z-score_cutoff'] = pd.to_numeric(final_df['z-score_cutoff'])

# Select and reorder the final columns
final_df = final_df[['District', 'stream', 'degree', 'year', 'students_sat', 'students_passed', 'z-score_cutoff']]

final_df.to_csv('final_dataset.csv', index=False)

print("Final Merged and Cleaned Data Head:")
final_df.head()

Final Merged and Cleaned Data Head:


,District,stream,degree,year,students_sat,students_passed,z-score_cutoff
0,colombo,biological science,MEDICINE University of Colombo,2019,2688,1838,2.4546
1,gampaha,biological science,MEDICINE University of Colombo,2019,1648,970,2.5010
2,kalutara,biological science,MEDICINE University of Colombo,2019,1018,609,2.5199
3,matale,biological science,MEDICINE University of Colombo,2019,443,179,2.5115
4,kandy,biological science,MEDICINE University of Colombo,2019,1615,860,2.5805


In [14]:
# 6. Feature Engineering and Encoding for Machine Learning

df = pd.read_csv('final_dataset.csv')

# --- Feature Engineering ---
# 1. Create pass_rate
# Replace 0s in students_sat to avoid division by zero, then calculate rate
df['pass_rate'] = df['students_passed'] / df['students_sat'].replace(0, 1) 

# 2. Create a time index from the year
df['time_index'] = df['year'] - df['year'].min()

# Define features and target. We will use the new engineered features.
features = ['District', 'stream', 'degree', 'time_index', 'pass_rate']
target = 'z-score_cutoff'

X = df[features]
y = df[target]

# --- One-Hot Encoding ---
categorical_cols = ['District', 'stream', 'degree']
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
X_encoded.columns = X_encoded.columns.str.replace(r'[\"\[\]\{\},:]', '', regex=True)

# Save the final feature-engineered and encoded dataset
final_encoded_df = pd.concat([X_encoded, y], axis=1)
final_encoded_df.to_csv('final_dataset_engineered_encoded.csv', index=False)

print("Feature-Engineered and Encoded Data Head (Ready for ML):")
final_encoded_df.head()

Feature-Engineered and Encoded Data Head (Ready for ML):


,time_index,pass_rate,District_anuradhapura,District_badulla,District_batticaloa,District_colombo,District_galle,District_gampaha,District_hambantota,District_jaffna,...,degree_SOCIAL WORK University of Jayewardenepura,degree_SOCIAL WORK University of Peradeniya,degree_SPEECH AND HEARING SCIENCES University of Kelaniya,degree_STATISTICS & OPERATIONS RESEARCH University of Peradeniya,degree_URBAN BIORESOURCES .1 University of Sri Jayewardenepura,degree_URBAN BIORESOURCES University of Sri Jayewardenepura,degree_VETERINARY SCIENCE University of Peradeniya,degree_VISUAL & TECHNOLOGICAL ARTS Swami Vipulananda Institute of Aesthetic Studies,degree_VISUAL ARTS University of the Visual & Performing Arts,z-score_cutoff
0,0,0.683780,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2.4546
1,0,0.588592,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,2.5010
2,0,0.598232,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2.5199
3,0,0.404063,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2.5115
4,0,0.532508,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2.5805


In [15]:
# 7. Train and Evaluate a Time-Series Model

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

# Load the feature-engineered and encoded dataset
df = pd.read_csv('final_dataset_engineered_encoded.csv')

# --- Time-Based Splitting ---
# Identify the latest year for the test set
latest_year_index = df['time_index'].max()

# Split the data based on the time_index
train_df = df[df['time_index'] < latest_year_index]
test_df = df[df['time_index'] == latest_year_index]

# Separate features (X) and target (y)
X_train = train_df.drop(columns=['z-score_cutoff'])
y_train = train_df['z-score_cutoff']
X_test = test_df.drop(columns=['z-score_cutoff'])
y_test = test_df['z-score_cutoff']

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

# --- Model Training ---
# Initialize and train the LightGBM Regressor
lgbm = lgb.LGBMRegressor(random_state=42)
lgbm.fit(X_train, y_train)

# --- Model Evaluation ---
# Make predictions on the test set
y_pred = lgbm.predict(X_test)

# Calculate and print evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE) on the test set: {mae:.4f}')
print(f'R-squared (R2) on the test set: {r2:.4f}')

Training data shape: (7801, 189)
Testing data shape: (2805, 189)
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001556 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 596
[LightGBM] [Info] Number of data points in the train set: 7801, number of used features: 177
[LightGBM] [Info] Start training from score 1.284264
Mean Absolute Error (MAE) on the test set: 0.1317
R-squared (R2) on the test set: 0.7852
